In [ ]:
# CLI Classifier
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pathlib

In [ ]:
DATA_DIR = 'dataset'
BATCH_SIZE = 32
IMG_SIZE = 224
RANDOM_SEED = 123

In [45]:
data_dir = pathlib.Path(DATA_DIR).resolve()
print(f"Scanning: {data_dir}")

Scanning: C:\Users\HP\Desktop\classification model\dataset


In [46]:
class_names = sorted([item.name for item in data_dir.glob('*') if item.is_dir()])
class_indices = {name: index for index, name in enumerate(class_names)}
print(f"Classes found: {class_names}")

Classes found: ['animals', 'humans', 'other']


In [47]:
extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp','*.webp','*.jfif','*.tiff']
all_image_paths = []
all_image_labels = []

In [48]:
print("Searching for images...")
for ext in extensions:
    for path in data_dir.rglob(ext):
        path_obj = pathlib.Path(path)
        try:
            relative_path = path_obj.relative_to(data_dir)
            main_class = relative_path.parts[0]
            if main_class in class_indices:
                all_image_paths.append(str(path))
                all_image_labels.append(class_indices[main_class])
        except ValueError:
            continue

print(f" Found {len(all_image_paths)} images total.")

Searching for images...
 Found 1799 images total.


In [49]:
train_paths, val_paths, train_labels, val_labels = train_test_split(
    all_image_paths, all_image_labels, 
    test_size=0.2, 
    random_state=RANDOM_SEED, 
    stratify=all_image_labels
)

In [50]:
def load_and_pad_image(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize_with_pad(img, IMG_SIZE, IMG_SIZE)
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    label = tf.one_hot(label, depth=len(class_names))
    return img, label

In [ ]:
def create_dataset(paths, labels, is_training=False):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(load_and_pad_image, num_parallel_calls=tf.data.AUTOTUNE)
    if is_training:
        ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

In [52]:
train_ds = create_dataset(train_paths, train_labels, is_training=True)
val_ds = create_dataset(val_paths, val_labels, is_training=False)

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),      
], name="augmentation_block")

In [67]:
inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_augmentation(inputs)

In [68]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=x)
base_model.trainable = False

C:\Users\HP\AppData\Local\Temp\ipykernel_12920\2933382203.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=x)


In [14]:
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)

In [15]:
x = layers.Dense(128)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Dropout(0.5)(x)

In [16]:
outputs = layers.Dense(len(class_names), activation='sigmoid')(x)

In [17]:
model = models.Model(inputs=inputs, outputs=outputs)

In [18]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [19]:
print("\nStarting Training (Frozen Base)...")
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


Starting Training (Frozen Base)...


In [20]:
history = model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=20, 
    callbacks=[early_stop]
)

Epoch 1/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 128s 2s/step - accuracy: 0.8353 - loss: 0.3437 - val_accuracy: 0.9194 - val_loss: 0.2065
Epoch 2/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 96s 1s/step - accuracy: 0.9527 - loss: 0.1390 - val_accuracy: 0.9556 - val_loss: 0.1345
Epoch 3/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 100s 2s/step - accuracy: 0.9715 - loss: 0.0953 - val_accuracy: 0.9667 - val_loss: 0.0815
Epoch 4/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 97s 1s/step - accuracy: 0.9729 - loss: 0.0793 - val_accuracy: 0.9722 - val_loss: 0.0701
Epoch 5/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 109s 2s/step - accuracy: 0.9701 - loss: 0.0699 - val_accuracy: 0.9556 - val_loss: 0.0740
Epoch 6/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 102s 2s/step - accuracy: 0.9778 - loss: 0.0603 - val_accuracy: 0.9778 - val_loss: 0.0623
Epoch 7/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 101s 2s/step - accuracy: 0.9826 - loss: 0.0472 - val_accuracy: 0.9722 - val_loss: 0.0668
Epoch 8/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 110s 2s/step - accuracy: 0.9798 - loss: 0.0502 - val_accuracy: 0.9722 - val

In [ ]:
model.save("sigmoid_model.keras")